# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [68]:
# import libraries
import sqlite3
import pandas as pd
import numpy as np
import scipy as sp
import re
from sqlalchemy import create_engine

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LocalAdmin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LocalAdmin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LocalAdmin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\LocalAdmin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [57]:
# load data from database
engine = create_engine('sqlite:///MessagesDB.db')
df = pd.read_sql_table('MessagesDB',engine)
X = df.message.values
y = df.drop(columns=df.columns[0:4],axis=1)


### 2. Write a tokenization function to process your text data

In [58]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)

    tokens = [WordNetLemmatizer().lemmatize(w).strip() for w in tokens if w not in stopwords.words("english")]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [63]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))
]) 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.3)
pipeline.fit(X_train , y_train)


c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000221BFA34AF0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [65]:
y_pred = pipeline.predict(X_test)

In [94]:
for col in range(y_pred.shape[1]):

    print(classification_report(y_test.iloc[:,col],y_pred[:,col]))


              precision    recall  f1-score   support

           0       0.71      0.42      0.53      1861
           1       0.84      0.94      0.89      5946
           2       0.24      0.43      0.31        58

    accuracy                           0.81      7865
   macro avg       0.60      0.60      0.57      7865
weighted avg       0.80      0.81      0.80      7865

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      6556
           1       0.81      0.52      0.63      1309

    accuracy                           0.90      7865
   macro avg       0.86      0.75      0.79      7865
weighted avg       0.89      0.90      0.89      7865

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0

c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\User

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      7610
           1       0.50      0.02      0.03       255

    accuracy                           0.97      7865
   macro avg       0.73      0.51      0.51      7865
weighted avg       0.95      0.97      0.95      7865

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      7514
           1       0.80      0.15      0.25       351

    accuracy                           0.96      7865
   macro avg       0.88      0.57      0.62      7865
weighted avg       0.95      0.96      0.95      7865

              precision    recall  f1-score   support

           0       0.87      1.00      0.93      6833
           1       0.48      0.03      0.06      1032

    accuracy                           0.87      7865
   macro avg       0.68      0.51      0.49      7865
weighted avg       0.82      0.87      0.81      7865

              preci

c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\User

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.